# Imports

In [2]:
import json
import emoji
import pandas as pd

# harcoded input variables.
## to be replaced by kwargs* or argv, argc

In [3]:
path = "/Users/sebastian.pizarro/0code/0nexus/aa-dec/data/farmers-protest-tweets-2021-2-4.json"

# Testing input file
## a json per line, splitted by '\n' within the file

In [4]:
from pprint import pprint
raw_data_list = []
with open(path) as fp:
    i = 0 
    for line in fp.readlines():
        line = line.strip()
        raw_data = json.loads(line)
        raw_data_list.append(raw_data)

In [5]:
def reload_df(raw_data_list):
    return pd.DataFrame(raw_data_list)


# Question defs

In [36]:
# 1. Los top 10 tweets más retweeted
# id, sum(retweenCount) order by 2 desc limit K

def q1(df):
    from pprint import pprint
    df = df[['id','retweetCount']].reindex()
    df=df.groupby('id')['retweetCount'].agg(MySum='sum')
    df = df.sort_values(by='MySum', ascending=False)
    pprint(df)
    df.info()
    
    
# 2. Los top 10 users en función a la cantidad de tweets que emitieron.
# user, count(id) order by 2 desc limit K
def q2(df):
    from pprint import pprint
    df = df[['user','id']]
    df['username'] = df['user'].apply(lambda x : x['username'])
    df = df[['username','id']].reindex()
    df = df.groupby('username')['id'].agg(MyCount='count')
    df = df.sort_values(by='MyCount', ascending=False)
    pprint(df)

    # 3. Los top 10 días donde hay más tweets. 
# date, count(id) order by 2 desc limit K
def q3(df):
    from pprint import pprint
    df = df[df["date"]!=""]
    df = df[df["date"].notnull()]
    df['date_object'] = pd.to_datetime(df['date'], format="%Y-%m-%dT%H:%M:%S+00:00")
    df['date_day'] =df['date_object'].apply( lambda dt : dt.strftime("%Y-%m-%d"))
    df = df [['date_day','id']]
    print(df.groupby('date_day').count())

# Los top 10 hashtags más usados.
def q4():
    def get_hashtags_array(text):
        """
        Check if a string contains an emoji.

        Args:
            text (str): The input text to check for emojis.

        Returns:
            bool: True if the input text contains an emoji, False otherwise.
        """
        words = text.split(' ')
        words = [word for word in words if words.contains("#")]
        words = words.apply(lambda w : w.strip())
        #return any(char in emoji.UNICODE_EMOJI for char in text)
        return words
    from pprint import pprint
    df["hashtags"] = df["content"].apply(lambda tweet: get_hashtags_array(tweet))
    df.info()


# 5. Los top 10 emojis más usados.
## search for character hex value within LOWER_LIMIT and UPPER_LMIT
## hex_value, count(*) order by 2 desc limit K
def q5(df):
    def has_emoji(text):
        """
        Check if a string contains an emoji.

        Args:
            text (str): The input text to check for emojis.

        Returns:
            bool: True if the input text contains an emoji, False otherwise.
        """
        response = list(emoji.analyze(text))
        d = []
        for e in response:
            i=0
            if e is None:
                pass
            d.append(({"emoji":e.value.emoji, "count": 1}))
            i=i+1
            if i > 10:
                break
        
        #print(d)
        return d


    #from pprint import pprint
    df["has_emoji"] = df["content"].apply(lambda tweet: has_emoji(tweet))
    #df.info()


# 6. Los top 10 users más influyentes en función de lo retweeted de sus tweets.
## user, count(id) where retweeted_count > 0
def q6(df):
    from pprint import pprint
    df = df.loc[df["retwee"]]
    df = df[['user','id']]

    df['username'] = df['user'].apply(lambda x : x['username'])
    df = df[['username','id']].reindex()
    df = df.groupby('username')['id'].agg(MyCount='count')
    df = df.sort_values(by='MyCount', ascending=False)
    pprint(df)




df=reload_df(raw_data_list)
q5(df)


KeyboardInterrupt: 